In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "0aStgWSO0E0E"
      },
      "source": [
        "# Regression"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "1eLEkw5O0ECa"
      },
      "source": [
        "## Objectives\n",
        "\n",
        "*   Fit and evaluate a regression model to predict tenure levels for a prospect that will likely churn\n",
        "\n",
        "\n",
        "## Inputs\n",
        "\n",
        "* outputs/datasets/collection/TelcoCustomerChurn.csv\n",
        "* Instructions on which variables to use for data cleaning and feature engineering. They are found in their respective notebooks.\n",
        "\n",
        "## Outputs\n",
        "\n",
        "* Train set (features and target)\n",
        "* Test set (features and target)\n",
        "* ML pipeline to predict tenure\n",
        "* labels map\n",
        "* Feature Importance Plot\n",
        "\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "9uWZXH9LwoQg"
      },
      "source": [
        "---"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# Change working directory"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "We need to change the working directory from its current folder to its parent folder\n",
        "* We access the current directory with os.getcwd()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import os\n",
        "current_dir = os.getcwd()\n",
        "current_dir"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "We want to make the parent of the current directory the new current directory\n",
        "* os.path.dirname() gets the parent directory\n",
        "* os.chir() defines the new current directory"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "os.chdir(os.path.dirname(current_dir))\n",
        "print(\"You set a new current directory\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "Confirm the new current directory"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "current_dir = os.getcwd()\n",
        "current_dir"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "pXKlJFX0iuM5"
      },
      "source": [
        "---"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "-mavJ8DibrcQ"
      },
      "source": [
        "# Load Data"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "Xk7DU_ekbtX8"
      },
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "import pandas as pd\n",
        "df = (pd.read_csv(\"outputs/datasets/collection/TelcoCustomerChurn.csv\")\n",
        "      .query(\"Churn == 1\")  # subset churned customer\n",
        "      .drop(labels=['customerID', 'TotalCharges', 'Churn'], axis=1)\n",
        "      # variables we will not need for this project\n",
        "      # we will not need Churn since it has only 1\n",
        "      )\n",
        "\n",
        "print(df.shape)\n",
        "df.head(3)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "krjAk78Tbyhv"
      },
      "source": [
        "# MP Pipeline: Regressor"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Create ML pipeline"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "C6keis6ao8LA"
      },
      "outputs": [],
      "source": [
        "from sklearn.pipeline import Pipeline\n",
        "\n",
        "# Feature Engineering\n",
        "from feature_engine.encoding import OrdinalEncoder\n",
        "from feature_engine.selection import SmartCorrelatedSelection\n",
        "\n",
        "# Feat Scaling\n",
        "from sklearn.preprocessing import StandardScaler\n",
        "\n",
        "# Feat Selection\n",
        "from sklearn.feature_selection import SelectFromModel\n",
        "\n",
        "# ML algorithms\n",
        "from sklearn.tree import DecisionTreeRegressor\n",
        "from xgboost import XGBRegressor\n",
        "from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor\n",
        "from sklearn.linear_model import LinearRegression\n",
        "from sklearn.ensemble import AdaBoostRegressor\n",
        "from sklearn.ensemble import ExtraTreesRegressor\n",
        "\n",
        "\n",
        "def PipelineOptimization(model):\n",
        "    pipeline_base = Pipeline([\n",
        "\n",
        "        (\"OrdinalCategoricalEncoder\", OrdinalEncoder(encoding_method='arbitrary',\n",
        "                                                     variables=['gender', 'Partner', 'Dependents', 'PhoneService',\n",
        "                                                                'MultipleLines', 'InternetService', 'OnlineSecurity',\n",
        "                                                                'OnlineBackup', 'DeviceProtection', 'TechSupport',\n",
        "                                                                'StreamingTV', 'StreamingMovies', 'Contract',\n",
        "                                                                'PaperlessBilling', 'PaymentMethod'])),\n",
        "\n",
        "\n",
        "        (\"SmartCorrelatedSelection\", SmartCorrelatedSelection(variables=None,\n",
        "         method=\"spearman\", threshold=0.6, selection_method=\"variance\")),\n",
        "\n",
        "        (\"feat_scaling\", StandardScaler()),\n",
        "\n",
        "        (\"feat_selection\",  SelectFromModel(model)),\n",
        "\n",
        "        (\"model\", model),\n",
        "\n",
        "    ])\n",
        "\n",
        "    return pipeline_base\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "lDmjjF3tHuCU"
      },
      "source": [
        "Custom Class for hyperparameter optimisation"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "NpTcVDtQ5RMc"
      },
      "outputs": [],
      "source": [
        "from sklearn.model_selection import GridSearchCV\n",
        "\n",
        "\n",
        "class HyperparameterOptimizationSearch:\n",
        "\n",
        "    def __init__(self, models, params):\n",
        "        self.models = models\n",
        "        self.params = params\n",
        "        self.keys = models.keys()\n",
        "        self.grid_searches = {}\n",
        "\n",
        "    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):\n",
        "        for key in self.keys:\n",
        "            print(f\"\\nRunning GridSearchCV for {key} \\n\")\n",
        "            model = PipelineOptimization(self.models[key])\n",
        "\n",
        "            params = self.params[key]\n",
        "            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,\n",
        "                              verbose=verbose, scoring=scoring)\n",
        "            gs.fit(X, y)\n",
        "            self.grid_searches[key] = gs\n",
        "\n",
        "    def score_summary(self, sort_by='mean_score'):\n",
        "        def row(key, scores, params):\n",
        "            d = {\n",
        "                'estimator': key,\n",
        "                'min_score': min(scores),\n",
        "                'max_score': max(scores),\n",
        "                'mean_score': np.mean(scores),\n",
        "                'std_score': np.std(scores),\n",
        "            }\n",
        "            return pd.Series({**params, **d})\n",
        "\n",
        "        rows = []\n",
        "        for k in self.grid_searches:\n",
        "            params = self.grid_searches[k].cv_results_['params']\n",
        "            scores = []\n",
        "            for i in range(self.grid_searches[k].cv):\n",
        "                key = \"split{}_test_score\".format(i)\n",
        "                r = self.grid_searches[k].cv_results_[key]\n",
        "                scores.append(r.reshape(len(params), 1))\n",
        "\n",
        "            all_scores = np.hstack(scores)\n",
        "            for p, s in zip(params, all_scores):\n",
        "                rows.append((row(k, s, p)))\n",
        "\n",
        "        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)\n",
        "\n",
        "        columns = ['estimator', 'min_score',\n",
        "                   'mean_score', 'max_score', 'std_score']\n",
        "        columns = columns + [c for c in df.columns if c not in columns]\n",
        "\n",
        "        return df[columns], self.grid_searches\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "LD6B3CuhiDMT"
      },
      "source": [
        "## Split Train Test Set"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "-pFzP2iGiIk1"
      },
      "outputs": [],
      "source": [
        "from sklearn.model_selection import train_test_split\n",
        "X_train, X_test, y_train, y_test = train_test_split(\n",
        "    df.drop(['tenure'], axis=1),\n",
        "    df['tenure'],\n",
        "    test_size=0.2,\n",
        "    random_state=0\n",
        ")\n",
        "\n",
        "print(\"* Train set:\", X_train.shape, y_train.shape,\n",
        "      \"\\n* Test set:\",  X_test.shape, y_test.shape)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "-15-sWUST6XX"
      },
      "source": [
        "## Grid Search CV - Sklearn"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "KTFXq-ieogBj"
      },
      "source": [
        "### Use default hyperparameters to find most suitable algorithm"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "XZKV86gsPw8c"
      },
      "outputs": [],
      "source": [
        "models_quick_search = {\n",
        "    'LinearRegression': LinearRegression(),\n",
        "    \"DecisionTreeRegressor\": DecisionTreeRegressor(random_state=0),\n",
        "    \"RandomForestRegressor\": RandomForestRegressor(random_state=0),\n",
        "    \"ExtraTreesRegressor\": ExtraTreesRegressor(random_state=0),\n",
        "    \"AdaBoostRegressor\": AdaBoostRegressor(random_state=0),\n",
        "    \"GradientBoostingRegressor\": GradientBoostingRegressor(random_state=0),\n",
        "    \"XGBRegressor\": XGBRegressor(random_state=0),\n",
        "}\n",
        "\n",
        "params_quick_search = {\n",
        "    'LinearRegression': {},\n",
        "    \"DecisionTreeRegressor\": {},\n",
        "    \"RandomForestRegressor\": {},\n",
        "    \"ExtraTreesRegressor\": {},\n",
        "    \"AdaBoostRegressor\": {},\n",
        "    \"GradientBoostingRegressor\": {},\n",
        "    \"XGBRegressor\": {},\n",
        "}"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "jGABtSoSLP9u"
      },
      "source": [
        "Do a hyperparameter optimisation search using default hyperparameters"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "-_q-ru92GiBb"
      },
      "outputs": [],
      "source": [
        "search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)\n",
        "search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "c7p56nXeoqWo"
      },
      "source": [
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "mq4YlrmZooiw"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "6pRUAeoG9lrZ"
      },
      "source": [
        "### Do an extensive search on the most suitable model to find the best hyperparameter configuration."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "w2XCyOYkAYpZ"
      },
      "source": [
        "Define model and parameters, for Extensive Search"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "lyjC7ThFAYKY"
      },
      "outputs": [],
      "source": [
        "models_search = {\n",
        "    \"GradientBoostingRegressor\": GradientBoostingRegressor(random_state=0),\n",
        "}\n",
        "\n",
        "# documentation to help on hyperparameter list: \n",
        "# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html\n",
        "\n",
        "# We will not conduct an extensive search, since the focus\n",
        "# is on how to combine all knowledge in an applied project.\n",
        "# In a workplace project, you may consider more hyperparameters and spend more time in this step\n",
        "\n",
        "params_search = {\n",
        "    \"GradientBoostingRegressor\": {\n",
        "        'model__n_estimators': [100,300],\n",
        "        'model__learning_rate': [1e-1,1e-2,1e-3], \n",
        "        'model__max_depth': [3,10,None],\n",
        "    }\n",
        "}"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "sBy8thxqAlrd"
      },
      "source": [
        "Extensive GridSearch CV"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "Y_4Ob7heAYM9"
      },
      "outputs": [],
      "source": [
        "search = HyperparameterOptimizationSearch(models=models_search, params=params_search)\n",
        "search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "wtNJJpLEAzdP"
      },
      "source": [
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "qjauRLNHAYPr"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "DWryh7BlA2df"
      },
      "source": [
        "Check the best model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "QVWEmpSuA4C7"
      },
      "outputs": [],
      "source": [
        "best_model = grid_search_summary.iloc[0, 0]\n",
        "best_model"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "7_jvnR4sZ8km"
      },
      "source": [
        "Parameters for best model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "2my-LZFzZ-YD"
      },
      "outputs": [],
      "source": [
        "grid_search_pipelines[best_model].best_params_"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "DgWXlprwaAW-"
      },
      "source": [
        "Define the best regressor, based on search"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "0OZ24jS0aAfP"
      },
      "outputs": [],
      "source": [
        "best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_\n",
        "best_regressor_pipeline"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "I9uT2XmaKISR"
      },
      "source": [
        "## Assess feature importance"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "-m6NUUa0KFQX"
      },
      "outputs": [],
      "source": [
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "sns.set_style('whitegrid')\n",
        "\n",
        "# after data cleaning and feature engineering, the features may have changes\n",
        "# how many data cleaning and feature engineering steps does your pipeline have?\n",
        "data_cleaning_feat_eng_steps = 2\n",
        "columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])\n",
        "                                        .transform(X_train)\n",
        "                                        .columns)\n",
        "\n",
        "best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support(\n",
        ")].to_list()\n",
        "\n",
        "# create DataFrame to display feature importance\n",
        "df_feature_importance = (pd.DataFrame(data={\n",
        "    'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()],\n",
        "    'Importance': best_regressor_pipeline['model'].feature_importances_})\n",
        "    .sort_values(by='Importance', ascending=False)\n",
        ")\n",
        "\n",
        "# Most important features statement and plot\n",
        "print(f\"* These are the {len(best_features)} most important features in descending order. \"\n",
        "      f\"The model was trained on them: \\n{df_feature_importance['Feature'].to_list()}\")\n",
        "\n",
        "df_feature_importance.plot(kind='bar', x='Feature', y='Importance')\n",
        "plt.show()\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "QzNyQirSKJj6"
      },
      "source": [
        "## Evaluate on Train and Test Sets"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "5pBm_vx8BO9s"
      },
      "outputs": [],
      "source": [
        "from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error\n",
        "import numpy as np\n",
        "\n",
        "\n",
        "def regression_performance(X_train, y_train, X_test, y_test, pipeline):\n",
        "    print(\"Model Evaluation \\n\")\n",
        "    print(\"* Train Set\")\n",
        "    regression_evaluation(X_train, y_train, pipeline)\n",
        "    print(\"* Test Set\")\n",
        "    regression_evaluation(X_test, y_test, pipeline)\n",
        "\n",
        "\n",
        "def regression_evaluation(X, y, pipeline):\n",
        "    prediction = pipeline.predict(X)\n",
        "    print('R2 Score:', r2_score(y, prediction).round(3))\n",
        "    print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))\n",
        "    print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))\n",
        "    print('Root Mean Squared Error:', np.sqrt(\n",
        "        mean_squared_error(y, prediction)).round(3))\n",
        "    print(\"\\n\")\n",
        "\n",
        "\n",
        "def regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline, alpha_scatter=0.5):\n",
        "    pred_train = pipeline.predict(X_train)\n",
        "    pred_test = pipeline.predict(X_test)\n",
        "\n",
        "    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))\n",
        "    sns.scatterplot(x=y_train, y=pred_train, alpha=alpha_scatter, ax=axes[0])\n",
        "    sns.lineplot(x=y_train, y=y_train, color='red', ax=axes[0])\n",
        "    axes[0].set_xlabel(\"Actual\")\n",
        "    axes[0].set_ylabel(\"Predictions\")\n",
        "    axes[0].set_title(\"Train Set\")\n",
        "\n",
        "    sns.scatterplot(x=y_test, y=pred_test, alpha=alpha_scatter, ax=axes[1])\n",
        "    sns.lineplot(x=y_test, y=y_test, color='red', ax=axes[1])\n",
        "    axes[1].set_xlabel(\"Actual\")\n",
        "    axes[1].set_ylabel(\"Predictions\")\n",
        "    axes[1].set_title(\"Test Set\")\n",
        "\n",
        "    plt.show()\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "tV-W5nYyBPdk"
      },
      "source": [
        "Evaluate Performance"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "EgBgrKJ5KFcX"
      },
      "outputs": [],
      "source": [
        "regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)\n",
        "regression_evaluation_plots(X_train, y_train, X_test, y_test, best_regressor_pipeline)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "HZ9tjLxEIn3h"
      },
      "source": [
        "# Regressor with PCA"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "Let's explore potential values for PCA n_components."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "pipeline = PipelineOptimization(model=LinearRegression())\n",
        "pipeline_pca = Pipeline(pipeline.steps[:3])\n",
        "df_pca = pipeline_pca.fit_transform(df.drop(['tenure'], axis=1))\n",
        "\n",
        "print(df_pca.shape,'\\n', type(df_pca))"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "Apply PCA separately to the scaled data"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "import numpy as np\n",
        "from sklearn.decomposition import PCA\n",
        "\n",
        "n_components = 17\n",
        "\n",
        "\n",
        "def pca_components_analysis(df_pca, n_components):\n",
        "    pca = PCA(n_components=n_components).fit(df_pca)\n",
        "    x_PCA = pca.transform(df_pca)  # array with transformed PCA\n",
        "\n",
        "    ComponentsList = [\"Component \" + str(number)\n",
        "                      for number in range(n_components)]\n",
        "    dfExplVarRatio = pd.DataFrame(\n",
        "        data=np.round(100 * pca.explained_variance_ratio_, 3),\n",
        "        index=ComponentsList,\n",
        "        columns=['Explained Variance Ratio (%)'])\n",
        "\n",
        "    dfExplVarRatio['Accumulated Variance'] = dfExplVarRatio['Explained Variance Ratio (%)'].cumsum(\n",
        "    )\n",
        "\n",
        "    PercentageOfDataExplained = dfExplVarRatio['Explained Variance Ratio (%)'].sum(\n",
        "    )\n",
        "\n",
        "    print(\n",
        "        f\"* The {n_components} components explain {round(PercentageOfDataExplained,2)}% of the data \\n\")\n",
        "    plt.figure(figsize=(12, 5))\n",
        "    sns.lineplot(data=dfExplVarRatio,  marker=\"o\")\n",
        "    plt.xticks(rotation=90)\n",
        "    plt.yticks(np.arange(0, 110, 10))\n",
        "    plt.show()\n",
        "\n",
        "\n",
        "pca_components_analysis(df_pca=df_pca, n_components=n_components)\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "n_components = 7\n",
        "pca_components_analysis(df_pca=df_pca, n_components=n_components)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "5eaMf41ZBhBk"
      },
      "source": [
        "## Rewrite ML Pipeline for Modelling"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "XfU562GBIsB1"
      },
      "outputs": [],
      "source": [
        "# PCA\n",
        "from sklearn.decomposition import PCA\n",
        "\n",
        "\n",
        "def PipelineOptimization(model):\n",
        "    pipeline_base = Pipeline([\n",
        "\n",
        "        (\"OrdinalCategoricalEncoder\", OrdinalEncoder(encoding_method='arbitrary',\n",
        "                                                     variables=['gender', 'Partner', 'Dependents', 'PhoneService',\n",
        "                                                                'MultipleLines', 'InternetService', 'OnlineSecurity',\n",
        "                                                                'OnlineBackup', 'DeviceProtection', 'TechSupport',\n",
        "                                                                'StreamingTV', 'StreamingMovies', 'Contract',\n",
        "                                                                'PaperlessBilling', 'PaymentMethod'])),\n",
        "\n",
        "\n",
        "        (\"SmartCorrelatedSelection\", SmartCorrelatedSelection(variables=None,\n",
        "         method=\"spearman\", threshold=0.6, selection_method=\"variance\")),\n",
        "\n",
        "\n",
        "        (\"feat_scaling\", StandardScaler()),\n",
        "\n",
        "        # PCA replace Feature Selection\n",
        "        (\"PCA\", PCA(n_components=7, random_state=0)),\n",
        "\n",
        "        (\"model\", model),\n",
        "\n",
        "    ])\n",
        "\n",
        "    return pipeline_base\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "irUsq475Bn7N"
      },
      "source": [
        "## Grid Search CV – Sklearn"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "print(\"* Train set:\", X_train.shape, y_train.shape, \"\\n* Test set:\",  X_test.shape, y_test.shape)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "2LVF-KR_Bqum"
      },
      "source": [
        "### Use standard hyperparameters to find the most suitable model."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "2XmJNoUcJkKX"
      },
      "outputs": [],
      "source": [
        "models_quick_search = {\n",
        "    'LinearRegression': LinearRegression(),\n",
        "    \"DecisionTreeRegressor\": DecisionTreeRegressor(random_state=0),\n",
        "    \"RandomForestRegressor\": RandomForestRegressor(random_state=0),\n",
        "    \"ExtraTreesRegressor\": ExtraTreesRegressor(random_state=0),\n",
        "    \"AdaBoostRegressor\": AdaBoostRegressor(random_state=0),\n",
        "    \"GradientBoostingRegressor\": GradientBoostingRegressor(random_state=0),\n",
        "    \"XGBRegressor\": XGBRegressor(random_state=0),\n",
        "}\n",
        "\n",
        "params_quick_search = {\n",
        "    'LinearRegression': {},\n",
        "    \"DecisionTreeRegressor\": {},\n",
        "    \"RandomForestRegressor\": {},\n",
        "    \"ExtraTreesRegressor\": {},\n",
        "    \"AdaBoostRegressor\": {},\n",
        "    \"GradientBoostingRegressor\": {},\n",
        "    \"XGBRegressor\": {},\n",
        "}\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Jq8td65fJkKY"
      },
      "source": [
        "Do a quick optimisation search "
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "s1BdqEB6JkKZ"
      },
      "outputs": [],
      "source": [
        "quick_search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)\n",
        "quick_search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "zQ_Xj5oGJkKZ"
      },
      "source": [
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "vIC2csxKJkKZ"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = quick_search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "383vOhJZBwza"
      },
      "source": [
        "### Do an extensive search on the most suitable model to find the best hyperparameter configuration."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "PrcbiQHlB9QT"
      },
      "source": [
        "Define model and parameters for extensive search"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "S7t-fum6B9QU"
      },
      "outputs": [],
      "source": [
        "models_search = {\n",
        "    \"GradientBoostingRegressor\":GradientBoostingRegressor(random_state=0),\n",
        "}\n",
        "\n",
        "# documentation to help on hyperparameter list: \n",
        "# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html\n",
        "\n",
        "# We will not conduct an extensive search, since the focus\n",
        "# is on how to combine all knowledge in an applied project.\n",
        "# In a workplace project, you may spend more time in this step\n",
        "params_search = {\n",
        "    \"GradientBoostingRegressor\":{\n",
        "        'model__n_estimators': [100,300],\n",
        "        'model__learning_rate': [1e-1,1e-2,1e-3], \n",
        "        'model__max_depth': [3,10, None],\n",
        "    }\n",
        "}"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Jvt-IOmHB9QU"
      },
      "source": [
        "Extensive GridSearch CV"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "eXbTwW1UB9QV"
      },
      "outputs": [],
      "source": [
        "search = HyperparameterOptimizationSearch(models=models_search, params=params_search)\n",
        "search.fit(X_train, y_train, scoring = 'r2', n_jobs=-1, cv=5)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "TVUDzRSGB9QV"
      },
      "source": [
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "5ZfOdh5kB9QW"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "CgoLo5C8B9QW"
      },
      "source": [
        "Check the best model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "C3UGYjpcB9QW"
      },
      "outputs": [],
      "source": [
        "best_model = grid_search_summary.iloc[0,0]\n",
        "best_model"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "bjY2MdBNB9QX"
      },
      "source": [
        "Parameters for best model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "Uspihv71B9QX"
      },
      "outputs": [],
      "source": [
        "grid_search_pipelines[best_model].best_params_"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "-8HonhniB9QX"
      },
      "source": [
        "Define the best regressor"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "DC8U4skKB9QY"
      },
      "outputs": [],
      "source": [
        "best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_\n",
        "best_regressor_pipeline"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "rKGmSgINCQwj"
      },
      "source": [
        "## Evaluate Regressor on Train and Tests Sets"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "78tt_ZkiJRdE"
      },
      "outputs": [],
      "source": [
        "regression_performance(X_train, y_train, X_test, y_test,best_regressor_pipeline)\n",
        "regression_evaluation_plots(X_train, y_train, X_test, y_test,\n",
        "                            best_regressor_pipeline)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "qpKz9qjRUOR0"
      },
      "source": [
        "# Convert Regression to Classification"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "g0yf7s9LVZFH"
      },
      "source": [
        "### Convert numerical target to bins, and check if it is balanced"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "dzXQjVf-Uoay"
      },
      "outputs": [],
      "source": [
        "from feature_engine.discretisation import EqualFrequencyDiscretiser\n",
        "disc = EqualFrequencyDiscretiser(q=3, variables=['tenure'])  # we will try q as 2, and 3\n",
        "df_clf = disc.fit_transform(df)\n",
        "\n",
        "print(f\"* The classes represent the following ranges: \\n{disc.binner_dict_} \\n\")\n",
        "sns.countplot(data=df_clf, x='tenure')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "df_clf.head(3)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "lkPaGZfiXexn"
      },
      "source": [
        "## Rewrite ML Pipeline for Modelling"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "I53mlfqRUSzn"
      },
      "outputs": [],
      "source": [
        "def PipelineOptimization(model):\n",
        "    pipeline_base = Pipeline([\n",
        "\n",
        "        (\"OrdinalCategoricalEncoder\", OrdinalEncoder(encoding_method='arbitrary',\n",
        "                                                     variables=['gender', 'Partner', 'Dependents', 'PhoneService',\n",
        "                                                                'MultipleLines', 'InternetService', 'OnlineSecurity',\n",
        "                                                                'OnlineBackup', 'DeviceProtection', 'TechSupport',\n",
        "                                                                'StreamingTV', 'StreamingMovies', 'Contract',\n",
        "                                                                'PaperlessBilling', 'PaymentMethod'])),\n",
        "\n",
        "\n",
        "        (\"SmartCorrelatedSelection\", SmartCorrelatedSelection(variables=None,\n",
        "         method=\"spearman\", threshold=0.6, selection_method=\"variance\")),\n",
        "\n",
        "        (\"feat_scaling\", StandardScaler()),\n",
        "\n",
        "        (\"feat_selection\",  SelectFromModel(model)),\n",
        "\n",
        "        (\"model\", model),\n",
        "\n",
        "    ])\n",
        "\n",
        "    return pipeline_base\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "fW2lu54NCgOC"
      },
      "source": [
        "## Load algorithms for classification"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "MvkMORvOft14"
      },
      "outputs": [],
      "source": [
        "from sklearn.tree import DecisionTreeClassifier \n",
        "from sklearn.ensemble import RandomForestClassifier\n",
        "from sklearn.ensemble import GradientBoostingClassifier \n",
        "from sklearn.ensemble import ExtraTreesClassifier\n",
        "from sklearn.ensemble import AdaBoostClassifier\n",
        "from xgboost import XGBClassifier"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "mVnTzZZBC73_"
      },
      "source": [
        "## Split Train Test Sets"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ev5Rb80fC-dP"
      },
      "outputs": [],
      "source": [
        "X_train, X_test, y_train, y_test = train_test_split(\n",
        "    df_clf.drop(['tenure'], axis=1),\n",
        "    df_clf['tenure'],\n",
        "    test_size=0.2,\n",
        "    random_state=0\n",
        ")\n",
        "\n",
        "print(\"* Train set:\", X_train.shape, y_train.shape,\n",
        "      \"\\n* Test set:\",  X_test.shape, y_test.shape)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "aTEEhpvRDF0a"
      },
      "source": [
        "## Grid Seach CV – Sklearn"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "PPNNBP-QDIkv"
      },
      "source": [
        "### Use standard hyper parameters to find most suitable model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "IfW5jSmSe7Gl"
      },
      "outputs": [],
      "source": [
        "models_quick_search = {\n",
        "    \"XGBClassifier\": XGBClassifier(random_state=0),\n",
        "    \"DecisionTreeClassifier\": DecisionTreeClassifier(random_state=0),\n",
        "    \"RandomForestClassifier\": RandomForestClassifier(random_state=0),\n",
        "    \"GradientBoostingClassifier\": GradientBoostingClassifier(random_state=0),\n",
        "    \"ExtraTreesClassifier\": ExtraTreesClassifier(random_state=0),\n",
        "    \"AdaBoostClassifier\": AdaBoostClassifier(random_state=0),\n",
        "}\n",
        "\n",
        "params_quick_search = {\n",
        "    \"XGBClassifier\":{},\n",
        "    \"DecisionTreeClassifier\":{},\n",
        "    \"RandomForestClassifier\":{},\n",
        "    \"GradientBoostingClassifier\":{},\n",
        "    \"ExtraTreesClassifier\":{},\n",
        "    \"AdaBoostClassifier\":{},\n",
        "}"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "YDYHs3I1D89F"
      },
      "source": [
        "GridSearch CV"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "dh-nd-JCfX7M"
      },
      "outputs": [],
      "source": [
        "from sklearn.metrics import make_scorer, recall_score\n",
        "quick_search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)\n",
        "quick_search.fit(X_train, y_train,\n",
        "                 scoring = make_scorer(recall_score, labels=[0], average=None),\n",
        "                 n_jobs=-1,\n",
        "                 cv=5)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "uqQfPBxfEQgf"
      },
      "source": [
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "yXUbzctLfXd2"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = quick_search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "uQWhAJtcDoSI"
      },
      "source": [
        "### Do an extensive search on the most suitable model to find the best hyperparameter configuration."
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "yyeaZbCFDxf4"
      },
      "source": [
        "Define models and parameters"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "FxEXnjBWDzOr"
      },
      "outputs": [],
      "source": [
        "models_search = {\n",
        "    \"AdaBoostClassifier\": AdaBoostClassifier(random_state=0),\n",
        "}\n",
        "\n",
        "# documentation to help on hyperparameter list:\n",
        "# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html\n",
        "params_search = {\n",
        "    \"AdaBoostClassifier\": {\n",
        "        'model__n_estimators': [50, 100, 300],\n",
        "        'model__learning_rate': [1e-1, 1e-2, 1e-3],\n",
        "    }\n",
        "}\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ZcqabQPpD_vX"
      },
      "source": [
        "Extensive GridSearch CV"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "CC2xgdgkECox"
      },
      "outputs": [],
      "source": [
        "from sklearn.metrics import make_scorer,  recall_score\n",
        "search = HyperparameterOptimizationSearch(\n",
        "    models=models_search, params=params_search)\n",
        "search.fit(X_train, y_train,\n",
        "           scoring=make_scorer(recall_score, labels=[0], average=None),\n",
        "           n_jobs=-1, cv=5)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "RDne0BgoEaDz"
      },
      "source": [
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "KsNiNwdGECra"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "-bZ2Qu5JEhrp"
      },
      "source": [
        "\n",
        "Check the best model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "aAbJosK8ECt-"
      },
      "outputs": [],
      "source": [
        "best_model = grid_search_summary.iloc[0,0]\n",
        "best_model"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "qUyqeVjkEjn7"
      },
      "source": [
        "Parameters for best model\n",
        "* We are saving this content for later"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "BXPyDbbxYbv6"
      },
      "outputs": [],
      "source": [
        "best_parameters = grid_search_pipelines[best_model].best_params_\n",
        "best_parameters"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "_YUeAvIsEo5v"
      },
      "source": [
        "Define the best clf pipeline"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "LTrUEOcBYby4"
      },
      "outputs": [],
      "source": [
        "pipeline_clf = grid_search_pipelines[best_model].best_estimator_\n",
        "pipeline_clf"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "aGc1W7wEM2GP"
      },
      "source": [
        "## Assess feature importance"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "fHN_VRwZarUp"
      },
      "source": [
        "We can assess feature importance for this model with `.feature_importances_`"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "kfckT_bFxFaE"
      },
      "outputs": [],
      "source": [
        "# after data cleaning and feat engine, the feature may space changes\n",
        "# how much data cleaning and feature engineering does your pipeline have?\n",
        "data_cleaning_feat_eng_steps = 2\n",
        "columns_after_data_cleaning_feat_eng = (Pipeline(pipeline_clf.steps[:data_cleaning_feat_eng_steps])\n",
        "                                        .transform(X_train)\n",
        "                                        .columns)\n",
        "\n",
        "# best_features = columns_after_data_cleaning_feat_eng\n",
        "best_features = columns_after_data_cleaning_feat_eng[pipeline_clf['feat_selection'].get_support(\n",
        ")].to_list()\n",
        "\n",
        "# create DataFrame to display feature importance\n",
        "df_feature_importance = (pd.DataFrame(data={\n",
        "    'Feature': columns_after_data_cleaning_feat_eng[pipeline_clf['feat_selection'].get_support()],\n",
        "    'Importance': pipeline_clf['model'].feature_importances_})\n",
        "    .sort_values(by='Importance', ascending=False)\n",
        ")\n",
        "\n",
        "# reassign best features in order\n",
        "best_features = df_feature_importance['Feature'].to_list()\n",
        "\n",
        "# Most important features statement and plot\n",
        "print(f\"* These are the {len(best_features)} most important features in descending order. \"\n",
        "      f\"The model was trained on them: \\n{best_features}\")\n",
        "\n",
        "df_feature_importance.plot(kind='bar', x='Feature', y='Importance')\n",
        "plt.show()\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "jiwI6U28UV4C"
      },
      "source": [
        "## Evaluate Classifier on Train and Test Sets"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "6sL9XamIDTG3"
      },
      "source": [
        "Custom Function"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "yMcvrPmdXbmP"
      },
      "outputs": [],
      "source": [
        "from sklearn.metrics import classification_report, confusion_matrix\n",
        "\n",
        "\n",
        "def confusion_matrix_and_report(X, y, pipeline, label_map):\n",
        "\n",
        "    prediction = pipeline.predict(X)\n",
        "\n",
        "    print('---  Confusion Matrix  ---')\n",
        "    print(pd.DataFrame(confusion_matrix(y_true=prediction, y_pred=y),\n",
        "          columns=[[\"Actual \" + sub for sub in label_map]],\n",
        "          index=[[\"Prediction \" + sub for sub in label_map]]\n",
        "          ))\n",
        "    print(\"\\n\")\n",
        "\n",
        "    print('---  Classification Report  ---')\n",
        "    print(classification_report(y, prediction, target_names=label_map), \"\\n\")\n",
        "\n",
        "\n",
        "def clf_performance(X_train, y_train, X_test, y_test, pipeline, label_map):\n",
        "    print(\"#### Train Set #### \\n\")\n",
        "    confusion_matrix_and_report(X_train, y_train, pipeline, label_map)\n",
        "\n",
        "    print(\"#### Test Set ####\\n\")\n",
        "    confusion_matrix_and_report(X_test, y_test, pipeline, label_map)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "PyX8PsH-0z8z"
      },
      "source": [
        "List that relates the classes and tenure interval"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "disc.binner_dict_['tenure']"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "We can create manually"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "label_map = ['<4.0', '4.0 to 20.0','+20.0']\n",
        "label_map"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "no7flbMcYbsz"
      },
      "outputs": [],
      "source": [
        "clf_performance(X_train=X_train, y_train=y_train,\n",
        "                        X_test=X_test, y_test=y_test,\n",
        "                        pipeline=pipeline_clf,\n",
        "                        label_map= label_map )"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "mQ3u0TodDdOZ"
      },
      "source": [
        "# Which pipeline to choose?"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "FE5va8Cr-CCy"
      },
      "source": [
        "We fitted 3 pipelines:\n",
        "* Regression\n",
        "* Regression with PCA\n",
        "* Classifier"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "sQR54xeCbIAH"
      },
      "source": [
        "The regressor pipelines didn't reach the expected performance threshold (0.7 R2 score) for the train and test set.\n",
        "\n",
        "The classifier was tuned on Recall for class 0 (tenure <4 months), since we are interested to detect prospects that may churn soon. \n",
        "* It has reasonable performance for class 0 (<4 months) and class 2 (+20 months)\n",
        "* Class 1 (4 to 20 months) has weak performance."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "pipeline_clf"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "1BqT1Kne54Fq"
      },
      "source": [
        "# Refit pipeline with best features"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "## Rewrite Pipeline"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "def PipelineOptimization(model):\n",
        "    pipeline_base = Pipeline([\n",
        "\n",
        "        (\"OrdinalCategoricalEncoder\", OrdinalEncoder(encoding_method='arbitrary',\n",
        "                                                     variables=['Contract', 'PaymentMethod'])),\n",
        "\n",
        "        (\"feat_scaling\", StandardScaler()),\n",
        "\n",
        "        # feature selection is not needed\n",
        "\n",
        "        (\"model\", model),\n",
        "\n",
        "    ])\n",
        "\n",
        "    return pipeline_base\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "GpgS-AgU6IWx"
      },
      "source": [
        "## Split Train Test Set, only with best features"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "_31XFcrg6IWy"
      },
      "outputs": [],
      "source": [
        "X_train, X_test, y_train, y_test = train_test_split(\n",
        "    df_clf.drop(['tenure'], axis=1),\n",
        "    df_clf['tenure'],\n",
        "    test_size=0.2,\n",
        "    random_state=0\n",
        ")\n",
        "\n",
        "print(\"* Train set:\", X_train.shape, y_train.shape,\n",
        "      \"\\n* Test set:\",  X_test.shape, y_test.shape)\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ohPWfCs2E_3G"
      },
      "source": [
        "Subset Best Features"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "UUEIfyLU6IWz"
      },
      "outputs": [],
      "source": [
        "X_train = X_train.filter(best_features)\n",
        "X_test = X_test.filter(best_features)\n",
        "\n",
        "print(\"* Train set:\", X_train.shape, y_train.shape, \"\\n* Test set:\",  X_test.shape, y_test.shape)\n",
        "X_train.head(3)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "2fT_mdLWFJFz"
      },
      "source": [
        "## Grid Search CV – Sklearn"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "RfKEBTyLeDtj"
      },
      "source": [
        "We are using the same model from the previous GridCV search"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "_1qcZktreHH5"
      },
      "outputs": [],
      "source": [
        "models_search"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "9WaSA9jcecXr"
      },
      "source": [
        "And the best parameters from the previous GridCV search"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "XXi0L025eKA6"
      },
      "outputs": [],
      "source": [
        "best_parameters"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "_7jAkvlBeeQl"
      },
      "source": [
        "You will need to type in manually since the hyperparameter values have to be a list. The previous dictionary is not in this format."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "u9HBXI2E58_5"
      },
      "outputs": [],
      "source": [
        "params_search = {'AdaBoostClassifier':  {\n",
        "    'model__learning_rate': [0.001],   # the value should be in []\n",
        "    'model__n_estimators': [50]       # the value should be in []\n",
        "}\n",
        "}\n",
        "params_search"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "zEZYXLRQfvTL"
      },
      "source": [
        "GridSearch CV"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "msJPkpo8fFAI"
      },
      "outputs": [],
      "source": [
        "from sklearn.metrics import make_scorer, recall_score\n",
        "search = HyperparameterOptimizationSearch(models=models_search, params=params_search)\n",
        "search.fit(X_train, y_train,\n",
        "           scoring = make_scorer(recall_score, labels=[0], average=None),\n",
        "           n_jobs=-1,cv=5)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "OcgDvuLRfwsE"
      },
      "source": [
        "\n",
        "Check results"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "loZEVp8g6q9O"
      },
      "outputs": [],
      "source": [
        "grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')\n",
        "grid_search_summary"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "3TE6Xgvif1ek"
      },
      "source": [
        "Check the best model"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "sf6qYXV06q9O"
      },
      "outputs": [],
      "source": [
        "best_model = grid_search_summary.iloc[0,0]\n",
        "best_model"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "oeB08Md3f60p"
      },
      "source": [
        "Define the best clf pipeline"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "YuA9mpyk6q9P"
      },
      "outputs": [],
      "source": [
        "pipeline_clf = grid_search_pipelines[best_model].best_estimator_\n",
        "pipeline_clf"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "IN0aj0iv6q9P"
      },
      "source": [
        "## Assess feature importance"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "dN-blGZb6q9P"
      },
      "outputs": [],
      "source": [
        "# how many data cleaning and feature engineering does your pipeline have?\n",
        "data_cleaning_feat_eng_steps = 1\n",
        "columns_after_data_cleaning_feat_eng = (Pipeline(pipeline_clf.steps[:data_cleaning_feat_eng_steps])\n",
        "                                        .transform(X_train)\n",
        "                                        .columns)\n",
        "\n",
        "best_features = columns_after_data_cleaning_feat_eng\n",
        "\n",
        "# create DataFrame to display feature importance\n",
        "df_feature_importance = (pd.DataFrame(data={\n",
        "    'Feature': columns_after_data_cleaning_feat_eng,\n",
        "    'Importance': pipeline_clf['model'].feature_importances_})\n",
        "    .sort_values(by='Importance', ascending=False)\n",
        ")\n",
        "\n",
        "# Most important features statement and plot\n",
        "print(f\"* These are the {len(best_features)} most important features in descending order. \"\n",
        "      f\"The model was trained on them: \\n{df_feature_importance['Feature'].to_list()}\")\n",
        "\n",
        "df_feature_importance.plot(kind='bar', x='Feature', y='Importance')\n",
        "plt.show()\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "h7fjgzReFYeM"
      },
      "source": [
        "## Evaluate Classifier on Train and Test Sets"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "aPeKtw3A59C3"
      },
      "outputs": [],
      "source": [
        "clf_performance(X_train=X_train, y_train=y_train,\n",
        "                        X_test=X_test, y_test=y_test,\n",
        "                        pipeline=pipeline_clf,\n",
        "                        label_map= label_map )"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "GBtppR73G1Yx"
      },
      "source": [
        "# Push files to the repo"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ShuJ5tYUC06o"
      },
      "source": [
        "We will generate the following files\n",
        "\n",
        "* Train set\n",
        "* Test set\n",
        "* Modeling pipeline\n",
        "* label map\n",
        "* features importance plot"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "8vBpPvnaG5Mb"
      },
      "outputs": [],
      "source": [
        "import joblib\n",
        "import os\n",
        "\n",
        "version = 'v1'\n",
        "file_path = f'outputs/ml_pipeline/predict_tenure/{version}'\n",
        "\n",
        "try:\n",
        "  os.makedirs(name=file_path)\n",
        "except Exception as e:\n",
        "  print(e)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "4TvoMsi3DNw1"
      },
      "source": [
        "## Train Set: features and target"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "yJHmwyqgDOr1"
      },
      "outputs": [],
      "source": [
        "X_train.head()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "yh6w6R7tDOvM"
      },
      "outputs": [],
      "source": [
        "X_train.to_csv(f\"{file_path}/X_train.csv\", index=False)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "pB6pjmAcDOym"
      },
      "outputs": [],
      "source": [
        "y_train"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "ZZ93HN6cDPBN"
      },
      "outputs": [],
      "source": [
        "y_train.to_csv(f\"{file_path}/y_train.csv\", index=False)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "aVbS3OnRDYtJ"
      },
      "source": [
        "## Test Set: features and target"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "XbgF38n1DaPp"
      },
      "outputs": [],
      "source": [
        "X_test.head()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "x9lM0xDvDaVZ"
      },
      "outputs": [],
      "source": [
        "X_test.to_csv(f\"{file_path}/X_test.csv\", index=False)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "5Jz66iMaDacI"
      },
      "outputs": [],
      "source": [
        "y_test"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "weYaJ4UxDake"
      },
      "outputs": [],
      "source": [
        "y_test.to_csv(f\"{file_path}/y_test.csv\", index=False)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "U-XpkYAPFncu"
      },
      "source": [
        "## Modelling pipeline"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "xLmFFWF6RGo6"
      },
      "source": [
        "ML pipeline for predicting tenure"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "vQkr4rcrHDnn"
      },
      "outputs": [],
      "source": [
        "pipeline_clf"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "YrZPif2aHdyO"
      },
      "outputs": [],
      "source": [
        "joblib.dump(value=pipeline_clf, filename=f\"{file_path}/clf_pipeline.pkl\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "LUCrXGvUFpeB"
      },
      "source": [
        "## List  mapping target levels to ranges"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "TFkAKp0eRMYM"
      },
      "source": [
        "Map for converting numerical variable to categorical variable"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "V6HfkzarHHbW"
      },
      "outputs": [],
      "source": [
        "label_map"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "TPEpdAgPHQaL"
      },
      "outputs": [],
      "source": [
        "joblib.dump(value=label_map, filename=f\"{file_path}/label_map.pkl\")"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "tTJlYRC5Q2wJ"
      },
      "source": [
        "## Feature importance plot"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "0SfLH05-Q2D8"
      },
      "outputs": [],
      "source": [
        "df_feature_importance.plot(kind='bar', x='Feature', y='Importance')\n",
        "plt.show()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "R-Hc2H3dQ74Z"
      },
      "outputs": [],
      "source": [
        "df_feature_importance.plot(kind='bar',x='Feature',y='Importance')\n",
        "plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Sh0SKfv_s-3V"
      },
      "source": [
        "Good job! Clear cell's outputs, push to the repo using git commands and move on to the next notebook"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "---"
      ]
    }
  ],
  "metadata": {
    "accelerator": "GPU",
    "colab": {
      "name": "Modeling and Evaluation - Predict Tenure.ipynb",
      "provenance": []
    },
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "language_info": {
      "codemirror_mode": {
        "name": "ipython",
        "version": 3
      },
      "file_extension": ".py",
      "mimetype": "text/x-python",
      "name": "python",
      "nbconvert_exporter": "python",
      "pygments_lexer": "ipython3",
      "version": "3.8.12 (default, Sep 27 2022, 15:56:02) \n[GCC 9.4.0]"
    },
    "orig_nbformat": 2,
    "vscode": {
      "interpreter": {
        "hash": "8b8334dab9339717f727a1deaf837b322d7a41c20d15cc86be99a8e69ceec8ce"
      }
    }
  },
  "nbformat": 4,
  "nbformat_minor": 2
}